In [1]:
import importlib
import sys
import re
import random
from urllib.request import urlopen
from urllib.request import Request

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
from pdfminer.pdfparser import PDFParser, PDFDocument
from PyPDF2 import PdfFileWriter, PdfFileReader

## 两个之前的函数

In [2]:
def  pdfCrap(path,start_page,i,save_path):
    """
    从pdf文件中截取几页，并保存在对应pdf文件中
    
    path:源文件的地址
    start_page:要开始的截取的页
    i:要截取的页数
    save_path:保存文件的地址
    """
    # 开始页
    start_page = start_page - 1
    # 截止页
    end_page = start_page +  i  # 这里设定截取几页
    output = PdfFileWriter()
    pdf_file = PdfFileReader(open(path, "rb"), strict=False)
    pdf_pages_len = pdf_file.getNumPages()
    for i  in  range(start_page, end_page):
        output.addPage(pdf_file.getPage(i)) # 在输出流中添加页
    outputStream =  open(save_path, "wb")
    output.write(outputStream)

In [25]:
def parse(path,text):
    '''
    将对应的pdf页转换成txt
    
    path：文件的地址
    text：保存的地址
    '''
    with open(path,'rb')as fp:# rb以二进制读模式打开本地pdf文件

        # 用文件对象来创建一个pdf文档分析器
        praser_pdf = PDFParser(fp)
        # 创建一个PDF文档
        doc = PDFDocument()

        # 连接分析器 与文档对象
        praser_pdf.set_document(doc)
        doc.set_parser(praser_pdf)

        # 提供初始化密码doc.initialize("123456")
        # 如果没有密码 就创建一个空的字符串
        doc.initialize()

        # 检测文档是否提供txt转换，不提供就忽略
        if not doc.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:
            # 创建PDf资源管理器 来管理共享资源
            rsrcmgr = PDFResourceManager()

            # 创建一个PDF参数分析器  创建一个PDF设备对象
            laparams = LAParams()

            # 创建聚合器
            device = PDFPageAggregator(rsrcmgr, laparams=laparams)

            # 创建一个PDF页面解释器对象
            interpreter = PDFPageInterpreter(rsrcmgr, device)

            # 循环遍历列表，每次处理一页的内容
            # doc.get_pages() 获取page列表
            for page in doc.get_pages():
                # 使用页面解释器来读取
                interpreter.process_page(page)

                # 使用聚合器获取内容
                layout = device.get_result()

                # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象 一般包括LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal 等等 想要获取文本就获得对象的text属性，
                for out in layout:
                    # 判断是否含有get_text()方法，图片之类的就没有
                    # if hasattr(out,"get_text"):
                    if isinstance(out, LTTextBoxHorizontal):
                        results = out.get_text()
                        text.write(results)


In [9]:
pdfCrap(r"F:\project\ML_project\Fintech100-2018-Report_Final_22-11-18sm.pdf",12,100,r"F:\project\ML_project\Fintech100-12-111.pdf")

## 把每一页pdf转成txt存储起来

In [2]:
text = []
with open(r"F:\project\ML_project\Fintech100-12-111.pdf",'rb')as fp:# rb以二进制读模式打开本地pdf文件

    # 用文件对象来创建一个pdf文档分析器
    praser_pdf = PDFParser(fp)
    # 创建一个PDF文档
    doc = PDFDocument()

    # 连接分析器 与文档对象
    praser_pdf.set_document(doc)
    doc.set_parser(praser_pdf)

    # 提供初始化密码doc.initialize("123456")
    # 如果没有密码 就创建一个空的字符串
    doc.initialize()

    # 检测文档是否提供txt转换，不提供就忽略
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        # 创建PDf资源管理器 来管理共享资源
        rsrcmgr = PDFResourceManager()

        # 创建一个PDF参数分析器  创建一个PDF设备对象
        laparams = LAParams()

        # 创建聚合器
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)

        # 创建一个PDF页面解释器对象
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # 循环遍历列表，每次处理一页的内容
        # doc.get_pages() 获取page列表
        for page in doc.get_pages():
            # 使用页面解释器来读取
            interpreter.process_page(page)

            # 使用聚合器获取内容
            layout = device.get_result()    
            results1 = ''
            for out in layout:
                if isinstance(out, LTTextBoxHorizontal):
                    results = out.get_text()
                    results1 = results1 + results
            text.append(results1)

In [3]:
text

['01\nAnt Financial\n“[Eric, CEO] has shown a unique combination of idealism, optimism and professionalism. He dreams big, \nyet is down-to-earth. He keeps his eye on the future while making every day count,” Jack Ma\nCompany Description\nAt a Glance\nAnt Financial Services Group is dedicated to using \ntechnology to bring the world equal opportunities. Their \ntechnologies, including blockchain, artificial intelligence, \nsercurity, Internet of Things and computing, has helped \nAnt Financial and their ecosystem partners to serve the \nunbanked and underbanked. \nBy the end of March 2018, together with their JV \npartners, Ant Financial served 870 million users \nworldwide. Brands under Ant Financial include Alipay, \nAnt Fortune, Yu’e Bao, Zhima Credit, MYbank and Ant \nFinancial Cloud.\nNotable Investors\nTemasek Holdings, GIC, China Investment Corp (CIC), CCB \nTrust, China Life, China Post Group, China Development \nBank Capital, Primavera Capital Group, National Social \nSecurity

## test pattern

In [51]:
patternOfName = re.compile(r'\d\d\n(.+)\n')

In [52]:
re.search(patternOfName,text[0]).group()

'01\nAnt Financial\n'

In [57]:
patternOfName.findall(text[0])[0]

'Ant Financial'

In [53]:
patternOfWebsite = re.compile(r'Website  \n(.+)\n')

In [54]:
re.search(patternOfWebsite,text[0]).group()

'Website  \nantgroup.com\n'

In [56]:
patternOfWebsite.findall(text[0])[0]

'antgroup.com'

In [58]:
patternOfYear = re.compile(r'Year Founded  \n(.+)\n')

In [59]:
patternOfYear.findall(text[0])[0]

'2004'

In [61]:
patternOfLocation = re.compile(r'Located  \n(.+)\n')

In [69]:
patternOfLocation.findall(text[0])[0]

'China'

In [23]:
patternOfInvestors = re.compile(r'Notable Investors\n([\s\S]+)\nCategory')

In [25]:
patternOfInvestors.findall(text[0])[0]

'Temasek Holdings, GIC, China Investment Corp (CIC), CCB \nTrust, China Life, China Post Group, China Development \nBank Capital, Primavera Capital Group, National Social \nSecurity Fund (NSSF)'

In [26]:
re.sub(r'\n',"",patternOfInvestors.findall(text[0])[0])

'Temasek Holdings, GIC, China Investment Corp (CIC), CCB Trust, China Life, China Post Group, China Development Bank Capital, Primavera Capital Group, National Social Security Fund (NSSF)'

In [27]:
patternOfDescription = re.compile(r'At a Glance\n([\s\S]+)\nNotable Investors')

In [28]:
patternOfDescription.findall(text[0])[0]

'Ant Financial Services Group is dedicated to using \ntechnology to bring the world equal opportunities. Their \ntechnologies, including blockchain, artificial intelligence, \nsercurity, Internet of Things and computing, has helped \nAnt Financial and their ecosystem partners to serve the \nunbanked and underbanked. \nBy the end of March 2018, together with their JV \npartners, Ant Financial served 870 million users \nworldwide. Brands under Ant Financial include Alipay, \nAnt Fortune, Yu’e Bao, Zhima Credit, MYbank and Ant \nFinancial Cloud.'

In [29]:
re.sub(r'\n',"",patternOfDescription.findall(text[0])[0])

'Ant Financial Services Group is dedicated to using technology to bring the world equal opportunities. Their technologies, including blockchain, artificial intelligence, sercurity, Internet of Things and computing, has helped Ant Financial and their ecosystem partners to serve the unbanked and underbanked. By the end of March 2018, together with their JV partners, Ant Financial served 870 million users worldwide. Brands under Ant Financial include Alipay, Ant Fortune, Yu’e Bao, Zhima Credit, MYbank and Ant Financial Cloud.'

In [30]:
patternOfCategory = re.compile(r'Category  \n(.+)')

In [31]:
patternOfCategory.findall(text[0])[0]

'Multi'

In [38]:
patternOfKeyPeople = re.compile(r'Key People  \n([\s\S]+)\nWebsite')

In [39]:
re.sub(r'\n',",",patternOfKeyPeople.findall(text[0])[0])

'Eric Jing'

In [40]:
patternOfOwnership  = re.compile(r'Ownership  \n(.+)')

In [41]:
patternOfOwnership.findall(text[0])[0]

'Private'

## get information

In [86]:
def get50LeadingCompanyInformation(text):
    patternOfName = re.compile(r'\d\d\n(.+)\n')
    patternOfDescription = re.compile(r'At a Glance\n([\s\S]+)\nNotable Investors')
    patternOfYear = re.compile(r'Year Founded  \n(.+)\n')
    patternOfLocation = re.compile(r'Located  \n(.+)\n')
    patternOfWebsite = re.compile(r'Website  \n(.+)\n')
    patternOfCategory = re.compile(r'Category  \n(.+)')
    patternOfOwnership  = re.compile(r'Ownership  \n(.+)')
    patternOfKeyPeople = re.compile(r'Key People  \n([\s\S]+)\nWebsite')
    patternOfInvestors = re.compile(r'Notable Investors\n([\s\S]+)\nCategory')
    try:
        Name = patternOfName.findall(text)[0]
    except:
        Name =None
    try:
        Description = re.sub(r'\n',"",patternOfDescription.findall(text)[0])
    except:
        Description = None
    try:
        Year = patternOfYear.findall(text)[0]
    except:
        Year = None
    try:
        Location = patternOfLocation.findall(text)[0]
    except:
        Location = None
    try:   
        Website = patternOfWebsite.findall(text)[0]
    except:
        Website = None
    try:
        Category = patternOfCategory.findall(text)[0]
    except:
        Category = None
    try:
        Ownership = patternOfOwnership.findall(text)[0]
    except:
        Ownership = None
    try:
        Keypeople = re.sub(r'\n',",",patternOfKeyPeople.findall(text)[0])
    except:
        Keypeople = None
    try:
        Investors = re.sub(r'\n',"",patternOfInvestors.findall(text)[0])
    except:
        Investors = None
    Information = {'Name': Name,'Description': Description,'Year': Year,'Location': Location,'Website': Website,'Category':Category,'Ownership':Ownership,'Keypeople':Keypeople,'Investors':Investors}
    return Information
    #return Name,Description,Year,Location,Website,Category,Ownership,Keypeople,Investors
    #return Information = {'Name': Name,'Description': Description,'Year': Year,'Location': Location,'Website': Website,'Category':Category,'Ownership':Ownership,'Keypeople':Keypeople,'Investors':Investors}

In [87]:
def get50EmergingCompanyInformation(text):
    patternOfName = re.compile(r'\ue801\n(.+)\n')
    patternOfDescription = re.compile(r'At a Glance\n([\s\S]+)\nNotable Investors')
    patternOfYear = re.compile(r'Year Founded  \n(.+)\n')
    patternOfLocation = re.compile(r'Located  \n(.+)\n')
    patternOfWebsite = re.compile(r'Website  \n(.+)\n')
    patternOfCategory = re.compile(r'Category  \n(.+)')
    patternOfOwnership  = re.compile(r'Ownership  \n(.+)')
    patternOfKeyPeople = re.compile(r'Key People  \n([\s\S]+)\nWebsite')
    patternOfInvestors = re.compile(r'Notable Investors\n([\s\S]+)\nCategory')
    try:
        Name = patternOfName.findall(text)[0]
    except:
        Name =None
    try:
        Description = re.sub(r'\n',"",patternOfDescription.findall(text)[0])
    except:
        Description = None
    try:
        Year = patternOfYear.findall(text)[0]
    except:
        Year = None
    try:
        Location = patternOfLocation.findall(text)[0]
    except:
        Location = None
    try:   
        Website = patternOfWebsite.findall(text)[0]
    except:
        Website = None
    try:
        Category = patternOfCategory.findall(text)[0]
    except:
        Category = None
    try:
        Ownership = patternOfOwnership.findall(text)[0]
    except:
        Ownership = None
    try:
        Keypeople = re.sub(r'\n',",",patternOfKeyPeople.findall(text)[0])
    except:
        Keypeople = None
    try:
        Investors = re.sub(r'\n',"",patternOfInvestors.findall(text)[0])
    except:
        Investors = None
    Information = {'Name': Name,'Description': Description,'Year': Year,'Location': Location,'Website': Website,'Category':Category,'Ownership':Ownership,'Keypeople':Keypeople,'Investors':Investors}
    return Information

In [88]:
Company100Information = []
a = 0
for i in text[0:50]:
    a += 1
    Company100Information.append(get50LeadingCompanyInformation(i))
    print(a)
for i in text[50:100]:
    a += 1
    Company100Information.append(get50EmergingCompanyInformation(i))
    print(a)
    #getCompanyInformation(i)
    #print(Name,Description,Year,Location,Website,Category,Ownership,Keypeople,Investors)
    #CompanyInformation = {'Name':Name,'Description':Description,'Year':Year,'Location':Location,'Website':Website,'Category':Category,'Ownership':Ownership,'Keypeople':Keypeople,'Investors':Investors}
   # Company100Information.append(Information)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [73]:
Company100Information

[{'Name': 'Ant Financial',
  'Description': 'Ant Financial Services Group is dedicated to using technology to bring the world equal opportunities. Their technologies, including blockchain, artificial intelligence, sercurity, Internet of Things and computing, has helped Ant Financial and their ecosystem partners to serve the unbanked and underbanked. By the end of March 2018, together with their JV partners, Ant Financial served 870 million users worldwide. Brands under Ant Financial include Alipay, Ant Fortune, Yu’e Bao, Zhima Credit, MYbank and Ant Financial Cloud.',
  'Year': '2004',
  'Location': 'China',
  'Website': 'antgroup.com',
  'Category': 'Multi',
  'Ownership': 'Private',
  'Keypeople': 'Eric Jing',
  'Investors': 'Temasek Holdings, GIC, China Investment Corp (CIC), CCB Trust, China Life, China Post Group, China Development Bank Capital, Primavera Capital Group, National Social Security Fund (NSSF)'},
 {'Name': 'JD Finance',
  'Description': None,
  'Year': '2013',
  'Loca

In [79]:
import json

In [90]:
with open('Company100Information.json', 'w') as f:
        json.dump(Company100Information, f, indent=4)

In [91]:
!start Company100Information.json

## test

In [2]:
import minecart

In [67]:
# minecart units are 1/72 inch, measured from bottom-left of the page
ROW_BORDERS = (
 72 * 0, # Bottom row starts 1 inch from the bottom of the page
 72 * 6.5, # Second row starts 3 inches from the bottom of the page
 72 * 11.125, # Third row starts 5 inches from the bottom of the page
 #72 * 7, # Third row ends 7 inches from the bottom of the page
)
COLUMN_BORDERS = (
 72 * 0, # Third col ends 8 inches from the left of the page
 72 * 4, # Third col starts 6 inches from the left of the page
 72 * 8, # Second col starts 4 inches from the left of the page 
 #72 * 2, # First col starts 2 inches from the left of the page
) # reversed so that BOXES is ordered properly
BOXES = [
 (left, bot, right, top)
 for top, bot in zip(ROW_BORDERS, ROW_BORDERS[1:])
 for left, right in zip(COLUMN_BORDERS, COLUMN_BORDERS[1:])
]

In [68]:
BOXES

[(0, 468.0, 288, 0),
 (288, 468.0, 576, 0),
 (0, 801.0, 288, 468.0),
 (288, 801.0, 576, 468.0)]

In [192]:
?minecart.Document.get_page

In [70]:
def extract_output(page):
    """
     Reads the text from page and splits it into the 9 cells.
     Returns a list with 9 entries: 
     [A, B, C, D, E, F, G, H, I]
     Each item in the tuple contains a string with all of the
     text found in the cell.
    """
    res = []
    for box in BOXES:
        strings = list(page.letterings.iter_in_bbox(box))
     # We sort from top-to-bottom and then from left-to-right, based
     # on the strings' top left corner
        strings.sort(key=lambda x: (-x.bbox[2], x.bbox[0]))
        res.append("".join(strings).replace(u"xa0","").strip())
    
    return res

In [196]:
a

'Company DescriptionJD Finance began operating as an independent company in October 2013. Positioned as a digital technology company serving financial institutions, JD Finance is dedicated to providing digital, online and offline all-scenario servicesaround three key points - data, user and connectivity, by means of such emerging technologies as big data, AI, cloud computing, blockchain and IoT. By so doing, it is aimed to empower financial institutions to improve efficiency, reduce costs and increase income in the core value sections like scenario expansion, customer acquisition, operation, risk control, and R&D, and propel the industry to comprehensively stride into the digital finance era.  Notable investors:CICC, BOCGI ,China Securities, CITIC Capital, Sequoia Capital China, China Harvest Investments and China Taiping Insurance.'

In [7]:
doc = minecart.Document(open("Fintech100-12-111.pdf", 'rb'))

a = ""
for i in doc.get_page(0).letterings.iter_in_bbox((30, 30, 300, 376)):
    a = a + i
print(a)
"""
for box in BOXES:
 strings = list(page.letterings.iter_in_bbox(box))
 # We sort from top-to-bottom and then from left-to-right, based
 # on the strings' top left corner
 strings.sort(key=lambda x: (-x.bbox[3], x.bbox[0]))
 res.append("".join(strings).replace(u"xa0","").strip())
"""   

KeyError: 'DeviceN'

In [215]:
doc.get_page(15)

AttributeError: 'PDFGraphicState' object has no attribute 'fill_color'

In [187]:
doc = minecart.Document(open("Fintech100-12-111.pdf", 'rb'))
b = ""
for i in doc.get_page(1).letterings.iter_in_bbox((300, 36, 576, 373)):
    b = b + i
print(b)

Tagline Creating a Better Future with TechnologyLocated  ChinaYear Founded  2013Key People  Shengqiang ChenWebsite  jr.jd.comCategory  MultiOwnership  PrivateRank Last Year  09Staff👤 👤 👤 👤 👤 At a Glance


In [134]:
b

'Tagline Creating a Better Future with TechnologyLocated  ChinaYear Founded  2013Key People  Shengqiang ChenWebsite  jr.jd.comCategory  MultiOwnership  PrivateRank Last Year  09Staff👤 👤 👤 👤 👤 At a Glance'

In [199]:
doc = minecart.Document(open("Fintech100-12-111.pdf", 'rb'))
#for page in doc.iter_pages():
 #   page.letterings.iter_in_bbox((300, 36, 576, 373))
doc

In [204]:
doc = minecart.Document(open("Fintech100-10-11.pdf", 'rb'))
doc.get_page(1)
#a = ""
#for i in doc.get_page(0).letterings:#.iter_in_bbox((36, 36, 300, 376)):
 #   a = a + i
#print(a)

AttributeError: 'PDFGraphicState' object has no attribute 'fill_color'

In [25]:
ROW_BORDERS = (
 72 * 1, # Bottom row starts 1 inch from the bottom of the page
 72 * 3, # Second row starts 3 inches from the bottom of the page
 72 * 5, # Third row starts 5 inches from the bottom of the page
 72 * 7, # Third row ends 7 inches from the bottom of the page
)
COLUMN_BORDERS = (
 72 * 8, # Third col ends 8 inches from the left of the page
 72 * 6, # Third col starts 6 inches from the left of the page
 72 * 4, # Second col starts 4 inches from the left of the page 
 72 * 2, # First col starts 2 inches from the left of the page
) # reversed so that BOXES is ordered properly
BOXES = [
 (left, bot, right, top)
 for top, bot in zip(ROW_BORDERS, ROW_BORDERS[1:])
 for left, right in zip(COLUMN_BORDERS, COLUMN_BORDERS[1:])
]

In [26]:
BOXES

[(576, 216, 432, 72),
 (432, 216, 288, 72),
 (288, 216, 144, 72),
 (576, 360, 432, 216),
 (432, 360, 288, 216),
 (288, 360, 144, 216),
 (576, 504, 432, 360),
 (432, 504, 288, 360),
 (288, 504, 144, 360)]

In [73]:
# minecart units are 1/72 inch, measured from bottom-left of the page
ROW_BORDERS = (
    72 * 1,  # Bottom row starts 1 inch from the bottom of the page
    72 * 3,  # Second row starts 3 inches from the bottom of the page
    72 * 5,  # Third row starts 5 inches from the bottom of the page
    72 * 7,  # Third row ends 7 inches from the bottom of the page
)
COLUMN_BORDERS = (
    72 * 8,  # Third col ends 8 inches from the left of the page
    72 * 6,  # Third col starts 6 inches from the left of the page
    72 * 4,  # Second col starts 4 inches from the left of the page   
    72 * 2,  # First col starts 2 inches from the left of the page
)  # reversed so that BOXES is ordered properly
BOXES = [
    (left, bot, right, top)
    for top, bot in zip(ROW_BORDERS, ROW_BORDERS[1:])
    for left, right in zip(COLUMN_BORDERS, COLUMN_BORDERS[1:])
]

def extract_output(page):
    """
    Reads the text from page and splits it into the 9 cells.

    Returns a list with 9 entries: 

        [A, B, C, D, E, F, G, H, I]

    Each item in the tuple contains a string with all of the
    text found in the cell.

    """
    res = []
    for box in BOXES:
        strings = list(page.letterings.iter_in_bbox(box))
        # We sort from top-to-bottom and then from left-to-right, based
        # on the strings' top left corner
        strings.sort(key=lambda x: (-x.bbox[3], x.bbox[0]))
        res.append(" ".join(strings).replace(u"\xa0", " ").strip())
    return res

content = []
doc = minecart.Document(open("Fintech100-12-111.pdf", 'rb'))

content.append(extract_output(doc.get_page(1)))

In [74]:
content

[['', '', '', '', '', '', '', '', '']]

In [39]:
>>> pdffile = open('test.pdf', 'rb')
>>> doc = minecart.Document(pdffile)
>>> page = doc.get_page(0)
>>> for shape in page.shapes.iter_in_bbox((576, 216, 432, 72)):
...     print(shape.path, shape.fill.color.as_rgb())

KeyError: 'DeviceN'

## 得到每一页内容

In [3]:
name_box = (0,688,288,835)
description_investor_box = (30, 30, 300, 376)
glance_box = (288,41,576,376)

In [29]:
doc = minecart.Document(open("Fintech100-12-111.pdf", 'rb'))

In [30]:
"".join(doc.get_page(0).letterings.iter_in_bbox(name_box))

'01Ant Financial'

In [31]:
doc = minecart.Document(open("Fintech100-12-111.pdf", 'rb'))
"".join(doc.get_page(0).letterings.iter_in_bbox(description_investor_box))

'Company DescriptionAnt Financial Services Group is dedicated to using technology to bring the world equal opportunities. Their technologies, including blockchain, artificial intelligence, sercurity, Internet of Things and computing, has helped Ant Financial and their ecosystem partners to serve the unbanked and underbanked. By the end of March 2018, together with their JV partners, Ant Financial served 870 million users worldwide. Brands under Ant Financial include Alipay, Ant Fortune, Yu’e Bao, Zhima Credit, MYbank and Ant Financial Cloud.Notable InvestorsTemasek Holdings, GIC, China Investment Corp (CIC), CCB Trust, China Life, China Post Group, China Development Bank Capital, Primavera Capital Group, National Social Security Fund (NSSF)'

In [32]:
doc = minecart.Document(open("Fintech100-12-111.pdf", 'rb'))
"".join(doc.get_page(0).letterings.iter_in_bbox(glance_box))

'Tagline Bring small and beautiful changes to the worldLocated  ChinaYear Founded  2004Key People  Eric JingWebsite  antgroup.comCategory  MultiOwnership  PrivateRank Last Year  01Staff👤 👤 👤 👤 👤 At a Glance'

In [33]:
doc

In [46]:
file = open("Fintech100-12-111.pdf", 'rb')
doc = minecart.Document(file)
# page_0 =doc.get_page(0)
# page_1 =doc.get_page(1)

In [18]:
"".join(page_0.letterings.iter_in_bbox(description_investor_box))

'Company DescriptionAnt Financial Services Group is dedicated to using technology to bring the world equal opportunities. Their technologies, including blockchain, artificial intelligence, sercurity, Internet of Things and computing, has helped Ant Financial and their ecosystem partners to serve the unbanked and underbanked. By the end of March 2018, together with their JV partners, Ant Financial served 870 million users worldwide. Brands under Ant Financial include Alipay, Ant Fortune, Yu’e Bao, Zhima Credit, MYbank and Ant Financial Cloud.Notable InvestorsTemasek Holdings, GIC, China Investment Corp (CIC), CCB Trust, China Life, China Post Group, China Development Bank Capital, Primavera Capital Group, National Social Security Fund (NSSF)'

In [19]:
"".join(page_0.letterings.iter_in_bbox(name_box))

'01Ant Financial'

In [21]:
"".join(page_0.letterings.iter_in_bbox(glance_box))

'Tagline Bring small and beautiful changes to the worldLocated  ChinaYear Founded  2004Key People  Eric JingWebsite  antgroup.comCategory  MultiOwnership  PrivateRank Last Year  01Staff👤 👤 👤 👤 👤 At a Glance'

In [37]:
name = []
a = 0
while a <100:
    page =doc.get_page(a)
    name.append("".join(page.letterings.iter_in_bbox(name_box)))
    a += 1

In [38]:
name

['01Ant Financial',
 '02JD Finance',
 '03Grab',
 '04Du Xiaoman Financial',
 '05SoFi',
 '06Oscar Health',
 '07Nubank',
 '08Robinhood',
 '09Atom Bank',
 '10Lufax Holding',
 '11OneConnnect (PingAn Yi',
 '1251 Credit Card',
 '13Revolut',
 '14Compass',
 '15Stripe',
 '16Clover Health',
 '17Adyen',
 '18Policybazaar',
 '19Klarna',
 '20ACORN O',
 '21Kreditech Holding',
 '22Monzo',
 '23WeLab',
 '24Number26 (N26)',
 '25WealthSimple',
 '26AfterPay Touch',
 '27Dianrong',
 '28VivaRepublica (Toss)',
 '29QUOINE',
 '30Kabbage',
 '31Affirm',
 '32OurCrowd',
 '33SolarisBank',
 '34Future Finance',
 '35Neyber',
 '36ZhongAn',
 '37TransferWise',
 '38Pushpay',
 '39League Inc.',
 '40Circle',
 '41Lendingkart',
 '42Opendoor',
 '43Metromile',
 '44Folio',
 '45Lendix',
 '46GuiaBolso',
 '47Starling Bank',
 '48Coinbase',
 '49Airwallex',
 '50Lemonade',
 '\ue801Agri Digital',
 '\ue801Anyfin',
 '\ue801Aqeed Technology',
 '\ue801Bankera',
 '\ue801Blackmoon Financial Gr',
 '\ue801BlockFi Lending',
 '\ue801Brex',
 '\ue801Ca

In [43]:
glance = []
a = 0
while a <100:
    page =doc.get_page(a)
    glance.append("".join(page.letterings.iter_in_bbox(glance_box)))
    a += 1

In [44]:
glance

['Tagline Bring small and beautiful changes to the worldLocated  ChinaYear Founded  2004Key People  Eric JingWebsite  antgroup.comCategory  MultiOwnership  PrivateRank Last Year  01Staff👤 👤 👤 👤 👤 At a Glance',
 'Tagline Creating a Better Future with TechnologyLocated  ChinaYear Founded  2013Key People  Shengqiang ChenWebsite  jr.jd.comCategory  MultiOwnership  PrivateRank Last Year  09Staff👤 👤 👤 👤 👤 At a Glance',
 'Tagline Go cashless. Pay seamlessly.Located  SingaporeYear Founded  2012Key People  Anthony TanWebsite  grab.comCategory  MultiOwnership  PrivateRank Last Year  New to the listStaff👤 👤 👤 👤 👤 At a Glance',
 'Tagline Xiaoxiao Finance Located  ChinaYear Founded  2015Key People  Zhu Guang  Website  jinrong.baidu.comCategory  MultiOwnership  PrivateRank Last Year  New to the listStaff👤 👤 👤 👤 👤 At a Glance',
 'Tagline A finance company that can help you get ahead Located  United StatesYear Founded  2011Key People  Anthony NotoWebsite  sofi.comCategory  LendingOwnership  PrivateRan

In [47]:
description_investor = []
a = 0
while a <100:
    page =doc.get_page(a)
    description_investor.append("".join(page.letterings.iter_in_bbox(description_investor_box)))
    a += 1

In [126]:
description_investor[42]

'Company DescriptionMetromile is revolutionizing car insurance through technology with its pay-per-mile insurance model. The company offers a device called the Metromile Pulse that plugs into a car to monitor its hours on the road. Based on this data, Metromile offers drivers tailored insurance plans. Last July the company debuted Ava, an automated assistant that uses car sensor data to verify insurance claims on the spot. Using data derived from Pulse, Ava reconstructs the scene of the accident to establish what happened and can automatically approve payments in the matter of seconds.Notable InvestorsTokio Marine, Holdings, Intact Financial Corporation, Section 32, Index Ventures,\xa0First Round Capital'

In [119]:
description_investor[0]

'Company DescriptionAnt Financial Services Group is dedicated to using technology to bring the world equal opportunities. Their technologies, including blockchain, artificial intelligence, sercurity, Internet of Things and computing, has helped Ant Financial and their ecosystem partners to serve the unbanked and underbanked. By the end of March 2018, together with their JV partners, Ant Financial served 870 million users worldwide. Brands under Ant Financial include Alipay, Ant Fortune, Yu’e Bao, Zhima Credit, MYbank and Ant Financial Cloud.Notable InvestorsTemasek Holdings, GIC, China Investment Corp (CIC), CCB Trust, China Life, China Post Group, China Development Bank Capital, Primavera Capital Group, National Social Security Fund (NSSF)'

In [120]:
patternOfDescription = re.compile(r'Company Description([\s\S]+)  Notable investors')
patternOfInvestors = re.compile(r'  Notable investors([\s\S]+)')

In [121]:
try:
    Description = patternOfDescription.findall(description_investor[1])[0]
except:
    Description = None

In [122]:
Description

'JD Finance began operating as an independent company in October 2013. Positioned as a digital technology company serving financial institutions, JD Finance is dedicated to providing digital, online and offline all-scenario servicesaround three key points - data, user and connectivity, by means of such emerging technologies as big data, AI, cloud computing, blockchain and IoT. By so doing, it is aimed to empower financial institutions to improve efficiency, reduce costs and increase income in the core value sections like scenario expansion, customer acquisition, operation, risk control, and R&D, and propel the industry to comprehensively stride into the digital finance era.'

In [123]:
try:
    Investors = patternOfInvestors.findall(description_investor[1])[0]
except:
    Investors = None

In [124]:
Investors

':CICC, BOCGI ,China Securities, CITIC Capital, Sequoia Capital China, China Harvest Investments and China Taiping Insurance.'

In [128]:
glance[26]

'Tagline The New Finance. For borrowers. For lenders. For you.Located  ChinaYear Founded  2012Key People Yuhang GuoSoul HtiteWebsite  dianrong.comCategory  LendingOwnership  PrivateRank Last Year  31Staff👤 👤 👤 👤 👤 At a Glance'

In [100]:
patternOfYear = re.compile(r'Year Founded  (.+)Key People')
patternOfLocation = re.compile(r'Located  (.+)Year Founded')
patternOfWebsite = re.compile(r'Website  (.+)Category')
patternOfCategory = re.compile(r'Category  (.+)Ownership')
patternOfOwnership  = re.compile(r'Ownership  (.+)Rank')
patternOfOwnershipafter = re.compile(r'Ownership  (.+)Staff')
patternOfKeyPeople = re.compile(r'Key People  ([\s\S]+)Website')
#rex = /([A-Z])([A-Z])([a-z])|([a-z])([A-Z])/g

In [106]:
    try:
        Year = patternOfYear.findall(glance[51])[0]
    except:
        Year = None
    try:
        Location = patternOfLocation.findall(glance[51])[0]
    except:
        Location = None
    try:
        Website = patternOfWebsite.findall(glance[51])[0]
    except:
        Website = None
    try:
        Category = patternOfCategory.findall(glance[51])[0]
    except:
        Category = None
    try:
        Ownership = patternOfOwnership.findall(glance[51])[0]
    except:
        if patternOfOwnershipafter.findall(glance[51]):#[0] != '':
            Ownership = patternOfOwnershipafter.findall(glance[51])[0]
        else:
            Ownership = None
    try:
        Keypeople = ','.join(re.split(r'(?<=[a-z])(?=[A-Z])', patternOfKeyPeople.findall(glance[51])[0]))
    except:
        Keypeople = None

In [91]:
Year

'2013'

In [92]:
Location

'United States'

In [93]:
Website

'www.robinhood.com'

In [94]:
Category

'Wealth'

In [107]:
Ownership

'Private'

In [98]:
Keypeople

'Baiju Bhatt,Vladimir Tenev,Nate Rodland'

In [87]:
''.join(re.split(r'(?<=[a-z])(?=[A-Z])', 'Mario SchlosserJoshua Kushner'))

'Mario Schlosser Joshua Kushner'